In [1]:
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import keras
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, population.shape[0]):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 2:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
all_geometry.shape

(38, 8, 3)

In [5]:
def fitness_function(pop,val):
    fitness=[0]*pop.shape[0]

    for i in range(pop.shape[0]):
        
        if (val[i][0][0][0]>=-2 and val[i][0][0][0]<=2) and (val[i][1][0][0]>=-2 and val[i][1][0][0]<=2):
            camber_fitness=abs(val[i][1][0][0]-val[i][0][0][0])/5
            fitness[i]+=camber_fitness
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][1]>=-1 and val[i][0][0][1]<=1) and (val[i][1][0][1]>=-1 and val[i][1][0][1]<=1):
            toe_fitness=abs(val[i][1][0][1]-val[i][0][0][1])/4
            fitness[i]+=toe_fitness
        else:
            fitness[i]=99999
            continue
        
        if (val[i][0][0][2]>=4 and val[i][0][0][2]<=9) and (val[i][1][0][2]>=4 and val[i][1][0][2]<=9):
            fitness[i]+=abs(val[i][1][0][2]-val[i][0][0][2])/2
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][3]>=3 and val[i][0][0][3]<=8) and (val[i][1][0][3]>=3 and val[i][1][0][3]<=8):
            fitness[i]+=abs(val[i][1][0][3]-val[i][0][0][3])/2
        else:
            fitness[i]=99999
            continue

    return fitness

In [6]:
#Use tournament selection

In [7]:
def tournament_selection(sp,pop,no_parents,tn_size,fitness_val):
    parents=np.empty([no_parents,pop.shape[1],pop.shape[2]])
    fitness_val=[1/(x+1e-4) for x in fitness_val]
    for i in range(no_parents):
        prob_tnt=[0]*tn_size
        select_index=random.sample([x for x in range(pop.shape[0])],tn_size)
        select_pop=pop[select_index]
        select_fitness=[fitness_val[x] for x in select_index]
        
        for j in range(tn_size):
            max_fitness_idx=select_fitness.index(max(select_fitness))
            prob_tnt[max_fitness_idx]=sp*((1-sp)**j)
            select_fitness[max_fitness_idx]=-9999999
            
        prob_tnt[tn_size-1]=(1-sp)**(tn_size-1)
        cdf=[prob_tnt[0]]
        
        for j in range(1,len(prob_tnt)):
            cdf.append(cdf[-1]+prob_tnt[j])
        
        selected=random.choices(select_pop,weights=cdf,k=1)
        parents[i]=np.array(selected[0])
    
    return parents

In [8]:
def crossover(parent,no_offspring,alpha):
    
    offspring=np.empty([no_offspring,parent.shape[1],parent.shape[2]])
    
    for i in range(no_offspring):
        parent1_index=i%parent.shape[0]
        parent2_index=(i+1)%parent.shape[0]
        n=random.random()
        offspring[i]=n*parent[parent1_index]+(1-n)*parent[parent2_index]
        
    return offspring

In [9]:
def mutation(offspring,mutation_rate):
    
    mutation=np.array(offspring,copy=True)
        
    for k in range(offspring.shape[0]):
        m=random.random()
        if m>mutation_rate:
            continue
            
        for j in range(offspring.shape[1]):
            for i in range(offspring.shape[2]):
                sign=random.randint(0,1)
                if sign==0:
                    mutation[k][j][i]=mutation[k][j][i]+random.random()
                else:
                    mutation[k][j][i]=mutation[k][j][i]-random.random()

    
    return mutation

In [10]:
def prepare_dataset(pop,model):
    X_flat=pop.reshape((pop.shape[0], -1))
    X=[]
    X_final=[]
    bump_val=[200,-60]
        
    for i in range(X_flat.shape[0]):
        for j in range(len(bump_val)):
            X.append(list(np.append(X_flat[i],bump_val[j])))
        #X_final.append(X)
        #X=[]
            
    X_final=np.array(X)
        
    train_df=pd.DataFrame(X_final)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_train = scaler.fit_transform(train_df)
    scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
        
    val=[]
    val_total=[]
    X = scaled_train_df[scaled_train_df.columns[:25]].values
    count=0
    for i in range(X.shape[0]):
        count+=1
        val1=model.predict(X[i].reshape(1,X[i].shape[0]))
        val.append(val1)
        if count==2:
            val_total.append(val)
            val=[]
            count=0
    return val_total

In [11]:
model=keras.models.load_model('GA_ANN_normal.h5')

In [12]:
def optimization(pop,no_generation,no_parent,mutation_rate):
    
    for m in range(no_generation):

        val_total=prepare_dataset(pop,model)
        pop_size=38
        no_offspring=pop_size-no_parent
        fitness=fitness_function(pop,val_total)
        parent=tournament_selection(0.9,pop,no_parent,10,fitness)
        offspring=crossover(parent,no_offspring,0.7)
        mutate=mutation(offspring,mutation_rate)
        pop=np.concatenate((pop,mutate),axis=0)
        
    return pop,fitness

In [13]:
pop,fitness=optimization(all_geometry,10,20,0.15)

In [14]:
index=fitness.index(min(fitness))

In [15]:
index

122

In [16]:
fitness[index]

0.11178777217864991

In [17]:
pop[index]

array([[3938.9511577116646, -180.36989893032992, 485.0704631413058],
       [4258.873010050327, -180.37836492697483, 455.09117227156014],
       [4081.916121510165, -464.93033312176414, 262.12375698907385],
       [3994.3200408175408, -185.38930559956202, 637.2126499538451],
       [4312.104196881783, -185.41288014406547, 607.1540392078421],
       [4098.369753897771, -453.9310494753264, 427.167063818065],
       [4031.5481910578246, -469.7633531658885, 325.24569053271654],
       [4000.7603654109002, -187.14352675183795, 536.371182652214]],
      dtype=object)

In [18]:
val_t=prepare_dataset(pop,model)

In [19]:
val_t[index]

[array([[-0.7423985,  0.6332642,  5.762166 ,  4.4459825]], dtype=float32),
 array([[-0.58019245,  0.6067089 ,  5.7016554 ,  4.4432845 ]],
       dtype=float32)]

In [18]:
fitness

[2.038229966163635,
 2.370081567764282,
 99999,
 99999,
 1.8543353915214538,
 2.0057116985321044,
 2.031228005886078,
 99999,
 99999,
 99999,
 99999,
 2.1076382756233216,
 99999,
 99999,
 99999,
 2.2249422907829284,
 99999,
 99999,
 2.076913332939148,
 0.48362255096435547,
 0.3285659313201904,
 0.34301109313964845,
 0.3930901885032654,
 0.28649147748947146,
 0.5857954025268555,
 0.6874512791633606,
 99999,
 2.038229966163635,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 2.278417730331421,
 99999,
 0.2786427140235901,
 0.26418629884719846,
 0.277182137966156,
 0.8472607016563416,
 2.158041310310364,
 99999,
 1.528547465801239,
 2.038229966163635,
 99999,
 99999,
 99999,
 0.27498801946640017,
 0.6647030591964722,
 0.4137355089187622,
 0.19202263355255128,
 0.6694063901901245,
 99999,
 99999,
 99999,
 0.5921844124794007,
 0.22893519401550294,
 2.349209415912628,
 2.0397318482398985,
 1.471735429763794,
 1.0536060452461242,
 99999,
 99999